In [13]:
# python -m http.server 8080
import requests
from bs4 import BeautifulSoup
import spacy
html = requests.get(url = 'http://localhost:8080/student.html').text
htmlParse = BeautifulSoup(html, 'html.parser').find_all('p')

text = ' '.join(x.string for x in htmlParse)


In [27]:
from spacy.matcher import Matcher 
def get_predicate(sent):
    
  doc = sent

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  # Define the pattern. 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"}] 
  
  matcher.add("matching_1", [pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  try: 
        span = doc[matches[k][1]:matches[k][2]] 
        predicate = span.text
  except IndexError:
        predicate = ''

  return(predicate)

In [292]:
def get_entities(sent):
  subj = ""           # Variable for storing the subject.
  pred = ""           # Variable for storing the object.
  obj = ""

  prv_tok_dep = ""    # Variable for dependency tag of previous token in the sentence. 
  prv_tok_text = ""   # Variable for previous token in the sentence.

  prefix = ""         # Variable for storing compounds.
  modifier = ""       # Variable for storing modifieres.

  # Loop through the tokens in the sentence.   
  for tok in sent:
    # Check if a token is a punctuation mark or not.
    if tok.dep_ != "punct":
      # Check if a token is a compound one or not.
      if tok.dep_ == "compound":
        # If yes, then store the token in the prefix variable.
        prefix = tok.text
        # Check if the previous token was also a compound one.
        if prv_tok_dep == "compound":
          # If yes, then update the prefix variable.
          prefix = prv_tok_text + " "+ tok.text
      
      # Check if a token is a modifier or not.
      if tok.dep_.endswith("mod") == True:
        # If yes, then store the token in the modifier varible.
        modifier = tok.text
         # Check if the previous token was a compound one.
        if prv_tok_dep == "compound":
        # If yes, then update the modifier variable.  
          modifier = prv_tok_text + " "+ tok.text
      
      # Check if a token is the subject.
      if tok.dep_.find("subj") == True:
        # If yes, then concatenate the modifier, prefix, and token
        # and assign the result to the subject variable (ent1).
        subj = modifier +" "+ prefix + " "+ tok.text
        # Reset the following variables: prefix, modifier, prv_tok_dep, and prv_tok_text.
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      # Check if a token is the object.
      if tok.dep_.find("obj") == True:
        # If yes, then concatenate the modifier, prefix, and token 
        # and assign the result to the object variable (ent2).    
        obj = modifier +" "+ prefix +" "+ tok.text
        
      # Update the variable for the dependency tag for the previous token. 
      prv_tok_dep = tok.dep_
      # Update the variable for the previous token in the sentence.  
      prv_tok_text = tok.text


 # Matcher class object 

  matcher = Matcher(nlp.vocab)

  # Define the pattern. 
  no_aux_pattern = [
            {'DEP':'aux', 'OP':"!"},
            {'DEP':'ROOT'}, 
            {'DEP':'det', 'OP': "?"},
            {'DEP':'attr', 'OP': "?"},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"}          
            ]
  aux_pattern = [
          {'DEP':'aux'},
          {'DEP':'ROOT'}, 
          {'DEP':'det', 'OP': "?"},
          {'DEP':'attr', 'OP': "?"},
          {'DEP':'prep','OP':"?"},
          {'DEP':'agent','OP':"?"}          
          ]
  auxpass_pattern = [
        {'DEP':'auxpass'},
        {'DEP':'ROOT'}, 
        {'DEP':'det', 'OP': "?"},
        {'DEP':'attr', 'OP': "?"},
        {'DEP':'prep','OP':"?"},
        {'DEP':'agent','OP':"?"}          
        ]

           # {'ROOT':'det'}] 
  
  matcher.add("no_aux", [no_aux_pattern]) 
  matcher.add("with_aux", [aux_pattern])
  matcher.add("with_auxpass", [auxpass_pattern])

  matches = matcher(sent)
  
  

  k = len(matches) - 1

  
 
  try: 
        k = len(matches) - 1
        start = matches[k][1]
        end = matches[k][2]
        pred = [] 
        for x in sent[start:end]:
              pred.append(x)

        print(pred)
        for token in pred:  
          print(token, token.dep_) 
          if(token.dep_ != 'aux' or token.dep_ != 'auxpass' or token.dep_ != 'ROOT'):
                pred.remove(token)
          
          if(token.dep_ == 'det'):
              pred.remove(token)  
          # if(nlp.vocab[match_id].text == 'no_aux'):
          #       pred = sent[start+1:end]
        # else:
        #   pred = sent[start:end]
       # span = sent[matches[k][1]:matches[k][2]]
        #print("span label: ", span.label)
        #pred = span.text
        
  except IndexError:
        pred = ''

  return [subj.strip(), pred, obj.strip()]

In [293]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
phrase_matcher = PhraseMatcher(nlp.vocab)

model = nlp(text)
sentances = []


for s in model.sents:
    sentances.append(s)
    
    
# for y in sentances:
#     print(get_entities(y))

# for current in sentances:
current = sentances[5]
print(get_entities(current))
# for x in current:
#          print(x,'->', x.dep_)
   

# 11840699188806025751


[is, studying, at]
is aux
at prep
['Kevin', [studying, at], 'Macquarie University']
